In [1]:
import argparse
import numpy as np
from scipy.io import wavfile

In [2]:
from audio.processor import WavProcessor, format_predictions

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
import csv
import os
import tensorflow as tf

from audio import params
from audio.utils import vggish, youtube8m

In [21]:
wav_file = "wav_files/sheep.wav"
sr, data = wavfile.read(wav_file)

FileNotFoundError: [Errno 2] No such file or directory: 'wav_files/shot distance9mm.53.wav'

In [10]:
proc = WavProcessor()

INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/youtube_model.ckpt


In [11]:
# Normalize (between -1 and 1)
samples = data / 32768.0

In [12]:
# Converts audio waveform into an array of examples for VGGish
"""Returns:
      3-D np.array of shape [num_examples, num_frames, num_bands] which represents
      a sequence of examples, each of which contains a patch of log mel
      spectrogram, covering num_frames frames of audio and num_bands mel frequency
      bands, where the frame length is params.STFT_HOP_LENGTH_SECONDS"""
examples_batch = vggish.input.waveform_to_examples(samples, sr)

In [13]:
examples_batch.shape

(2, 96, 64)

In [14]:
features = proc._get_features(examples_batch)

In [15]:
features.shape

(2, 128)

In [16]:
predictions = proc._process_features(features)

In [17]:
predictions.shape

(1, 527)

In [18]:
count = params.PREDICTIONS_COUNT_LIMIT
hit = params.PREDICTIONS_HIT_LIMIT

top_indices = np.argpartition(predictions[0], -count)[-count:]
line = ((proc._class_map[i], float(predictions[0][i])) for
                i in top_indices if predictions[0][i] > hit)
#        return sorted(line, key=lambda p: -p[1])

In [19]:
predictions = proc._filter_predictions(predictions)

In [20]:
predictions

[('Speech', 0.903878390789032),
 ('Crowing, cock-a-doodle-doo', 0.6612930297851562),
 ('Cough', 0.3694608807563782)]

In [ ]:
with open(params.CLASS_LABELS_INDICES) as f:
    next(f)  # skip header
    reader = csv.reader(f)
    for row in reader:
        print(row)
        #self._class_map[int(row[0])] = row[2]